In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
n_qubits = 4
n_layers = 1
dev = qml.device("default.qubit", wires=n_qubits)

In [9]:
# fix this garbage
def Hamiltonian(n_qubits):
    if n_qubits == 0:
        return None
    if n_qubits == 1:
        return qml.prod(
            qml.PauliZ(n_qubits - 1),
            qml.PauliZ(n_qubits - 1),
            qml.PauliX(n_qubits - 1)
        )
    return Hamiltonian(n_qubits - 1) @ qml.prod(
            qml.PauliZ(n_qubits - 1),
            qml.PauliZ(n_qubits - 1),
            qml.PauliX(n_qubits - 1)
        )

In [12]:
@qml.qnode(dev)
def TFIM(weights):
    for i in range(n_layers):
        qml.IsingZZ(weights[0], wires=[0,1])
        qml.IsingZZ(weights[0], wires=[2,3])
        qml.IsingZZ(weights[1], wires=[0,3]) # if "open boundary conditions" this gate isn't there
        qml.IsingZZ(weights[1], wires=[1,2])

        for i in range(n_qubits):
            qml.PauliX(i)
            # qml.PauliZ(i) # LTFIM
    return qml.expval(Hamiltonian(n_qubits))

weights = [np.random.random() for _ in range(2)]
print(qml.draw(TFIM)(weights))

0: ─╭IsingZZ(0.50)─╭IsingZZ(0.23)──X────────────────┤ ╭<(((Z@Z@X)@(Z@Z@X))@(Z@Z@X))@(Z@Z@X)>
1: ─╰IsingZZ(0.50)─│──────────────╭IsingZZ(0.23)──X─┤ ├<(((Z@Z@X)@(Z@Z@X))@(Z@Z@X))@(Z@Z@X)>
2: ─╭IsingZZ(0.50)─│──────────────╰IsingZZ(0.23)──X─┤ ├<(((Z@Z@X)@(Z@Z@X))@(Z@Z@X))@(Z@Z@X)>
3: ─╰IsingZZ(0.50)─╰IsingZZ(0.23)──X────────────────┤ ╰<(((Z@Z@X)@(Z@Z@X))@(Z@Z@X))@(Z@Z@X)>


In [ ]:
def cost_fn(param):
    

In [13]:
H = qml.Hamiltonian(
    [1, 1, 0.5],
    [qml.PauliX(0), qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1)]
)
print(H)

  (1) [X0]
+ (1) [Z1]
+ (0.5) [X0 X1]
